## Exploratory Analysis of Census Income Data
* Objective: Predict whether income exceeds $50K/yr based on census data, also known as Adult dataset
* Data cleanup approach: TBD
* Machine Learning Models: TBD

### Source of information: 
* https://archive.ics.uci.edu/dataset/20/census+income



## Part 1 - Install and import the required libraries

In [101]:
# Import required dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt
import numpy as np
import warnings
from sklearn.metrics import balanced_accuracy_score
warnings.filterwarnings("ignore")

import seaborn as sns

## Step 2 - Data Preparation
* Upload all the source files into /resources folder and add column headers: 
  * adult.data - which is used as the train data
  * adult.test - which is used as the test data
* Check the tables to understand data types, missing values, and fix them for better data quality

In [102]:
# Import the data
census_df = pd.read_csv('./resources/adult.data')
census_test_df = pd.read_csv('./resources/adult.test', skiprows=1) # skip the first row as it might contain headers


In [103]:
# Display the first 10 rows of the data
display(census_df.head(10))
display(census_test_df.head(10))

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
5,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
6,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
7,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
8,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K
9,37,Private,280464,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,80,United-States,>50K


,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0.1,40,United-States,<=50K.
0,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
1,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
2,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
3,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.
4,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K.
5,29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0,0,40,United-States,<=50K.
6,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K.
7,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K.
8,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K.
9,65,Private,184454,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,6418,0,40,United-States,>50K.


#### This section is for the census_df train datasets

In [104]:
# Add labels to the columns of the census_df
census_df.columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'
]

census_df.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
5,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
6,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
7,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
8,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K
9,37,Private,280464,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,80,United-States,>50K


In [105]:
# Check how many rows of census_df
census_df.shape

(32560, 15)

In [106]:
# Describe census_df
census_df.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32560.000000,3.256000e+04,32560.000000,32560.000000,32560.000000,32560.000000
mean,38.581634,1.897818e+05,10.080590,1077.615172,87.306511,40.437469
std,13.640642,1.055498e+05,2.572709,7385.402999,402.966116,12.347618
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178315e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783630e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370545e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [107]:
# Check for missing values
census_df.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64

In [108]:
# Check unique values in the each column
for column in census_df.columns:
    print(column)
    print(census_df[column].unique())
    print()

age
[50 38 53 28 37 49 52 31 42 30 23 32 40 34 25 43 54 35 59 56 19 39 20 45
 22 48 21 24 57 44 41 29 18 47 46 36 79 27 67 33 76 17 55 61 70 64 71 68
 66 51 58 26 60 90 75 65 77 62 63 80 72 74 69 73 81 78 88 82 83 84 85 86
 87]

workclass
[' Self-emp-not-inc' ' Private' ' State-gov' ' Federal-gov' ' Local-gov'
 ' ?' ' Self-emp-inc' ' Without-pay' ' Never-worked']

fnlwgt
[ 83311 215646 234721 ...  34066  84661 257302]

education
[' Bachelors' ' HS-grad' ' 11th' ' Masters' ' 9th' ' Some-college'
 ' Assoc-acdm' ' Assoc-voc' ' 7th-8th' ' Doctorate' ' Prof-school'
 ' 5th-6th' ' 10th' ' 1st-4th' ' Preschool' ' 12th']

education-num
[13  9  7 14  5 10 12 11  4 16 15  3  6  2  1  8]

marital-status
[' Married-civ-spouse' ' Divorced' ' Married-spouse-absent'
 ' Never-married' ' Separated' ' Married-AF-spouse' ' Widowed']

occupation
[' Exec-managerial' ' Handlers-cleaners' ' Prof-specialty'
 ' Other-service' ' Adm-clerical' ' Sales' ' Craft-repair'
 ' Transport-moving' ' Farming-fishing' ' Mac

#### Format 'marital-status' column data to binary

In [109]:
# Convert ' Divorced', ' Never-married', and ' Widowed' to '0' 
# and ' Married-civ-spouse', ' Married-AF-spouse', ' Married-spouse-absent' and ' Separated' to '1'
census_df['marital-status'] = census_df['marital-status'].replace({
    ' Divorced': 0, 
    ' Never-married': 0,  
    ' Widowed': 0,
    ' Married-civ-spouse': 1, 
    ' Married-AF-spouse': 1, 
    ' Married-spouse-absent': 1,
    ' Separated': 1
})

census_df.sample(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
22331,36,Private,152621,Some-college,10,1,Craft-repair,Husband,White,Male,0,0,40,Canada,>50K
10738,33,Private,80058,HS-grad,9,0,Other-service,Own-child,White,Male,0,0,48,United-States,<=50K
4809,40,Private,179717,Bachelors,13,0,Sales,Not-in-family,White,Male,0,1564,60,United-States,>50K
21585,29,?,116820,HS-grad,9,0,?,Own-child,White,Male,0,0,50,United-States,<=50K
16988,50,Private,104501,Masters,14,1,Prof-specialty,Husband,White,Male,0,0,40,United-States,>50K


#### Format 'income' column data to binary

In [110]:
# Convert ' <=50K' to '0' and ' >50K' to '1'
census_df['income'] = census_df['income'].replace({
    ' <=50K': 0, 
    ' >50K': 1
})

census_df.sample(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
28523,73,Private,349347,7th-8th,4,1,Handlers-cleaners,Husband,White,Male,0,0,25,United-States,0
17876,55,Private,240988,9th,5,1,Machine-op-inspct,Other-relative,Asian-Pac-Islander,Male,0,0,40,Philippines,0
20923,30,Private,154882,Prof-school,15,0,Other-service,Not-in-family,White,Male,0,0,35,United-States,0
15372,17,Private,95799,11th,7,0,Sales,Own-child,White,Female,0,0,18,United-States,0
16933,42,Self-emp-not-inc,140474,Assoc-acdm,12,0,Craft-repair,Own-child,Amer-Indian-Eskimo,Male,0,0,35,United-States,0


In [111]:
# The workclass, occupation, and native-country columns have a value '?' which means it is a missing value. 
# Note some of these ? have extra spaces around them
# We will replace it with a NaN value
census_df['workclass'] = census_df['workclass'].replace(r'^\s*\?\s*$', np.nan, regex=True)
census_df['occupation'] = census_df['occupation'].replace(r'^\s*\?\s*$', np.nan, regex=True)
census_df['native-country'] = census_df['native-country'].replace(r'^\s*\?\s*$', np.nan, regex=True)

In [112]:
# Drop all records that contain missing values (NaN)
census_df.dropna(inplace=True)

In [113]:
# Check how many rows of census_df
census_df.shape

(30161, 15)

##### For the Train datasets, we remove 2399 null values that originally had ? sign.
##### We now work with 30,161 rows from originally 32,560 rows

#### This section is for the census_test_df test datasets

In [114]:
# Add labels to the columns of the census_test_df
census_test_df.columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'
]

census_test_df.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
1,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
2,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
3,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.
4,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K.
5,29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0,0,40,United-States,<=50K.
6,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K.
7,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K.
8,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K.
9,65,Private,184454,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,6418,0,40,United-States,>50K.


In [115]:
# Check how many rows of census_test_df
census_test_df.shape

(16280, 15)

In [116]:
# Describe census_test_df
census_test_df.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,16280.000000,1.628000e+04,16280.000000,16280.00000,16280.000000,16280.000000
mean,38.768305,1.894334e+05,10.073096,1081.97156,87.904668,40.392260
std,13.849192,1.057177e+05,2.567511,7584.16416,403.117078,12.479715
min,17.000000,1.349200e+04,1.000000,0.00000,0.000000,1.000000
25%,28.000000,1.167162e+05,9.000000,0.00000,0.000000,40.000000
50%,37.000000,1.778295e+05,10.000000,0.00000,0.000000,40.000000
75%,48.000000,2.383840e+05,12.000000,0.00000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.00000,3770.000000,99.000000


In [117]:
# Check for missing values
census_test_df.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64

In [118]:
# Check unique values in the each column
for column in census_test_df.columns:
    print(column)
    print(census_test_df[column].unique())
    print()

age
[38 28 44 18 34 29 63 24 55 65 36 26 58 48 43 20 37 40 72 25 45 22 23 54
 32 46 56 17 39 52 21 42 33 30 47 41 19 69 50 31 59 49 51 27 57 61 64 79
 73 53 77 80 62 35 68 66 75 60 67 71 70 90 81 74 78 82 83 85 76 84 89 88
 87]

workclass
[' Private' ' Local-gov' ' ?' ' Self-emp-not-inc' ' Federal-gov'
 ' State-gov' ' Self-emp-inc' ' Without-pay' ' Never-worked']

fnlwgt
[ 89814 336951 160323 ... 349230 321403  83891]

education
[' HS-grad' ' Assoc-acdm' ' Some-college' ' 10th' ' Prof-school'
 ' 7th-8th' ' Bachelors' ' Masters' ' Doctorate' ' 11th' ' 5th-6th'
 ' Assoc-voc' ' 9th' ' 12th' ' 1st-4th' ' Preschool']

education-num
[ 9 12 10  6 15  4 13 14 16  7  3 11  5  8  2  1]

marital-status
[' Married-civ-spouse' ' Never-married' ' Widowed' ' Divorced'
 ' Separated' ' Married-spouse-absent' ' Married-AF-spouse']

occupation
[' Farming-fishing' ' Protective-serv' ' Machine-op-inspct' ' ?'
 ' Other-service' ' Prof-specialty' ' Craft-repair' ' Adm-clerical'
 ' Exec-managerial' ' Tech-sup

#### Format 'marital-status' column data to binary

In [119]:
# Convert ' Divorced', ' Never-married', and ' Widowed' to '0' 
# and ' Married-civ-spouse', ' Married-AF-spouse', ' Married-spouse-absent' and ' Separated' to '1'
census_test_df['marital-status'] = census_test_df['marital-status'].replace({
    ' Divorced': 0, 
    ' Never-married': 0,  
    ' Widowed': 0,
    ' Married-civ-spouse': 1, 
    ' Married-AF-spouse': 1, 
    ' Married-spouse-absent': 1,
    ' Separated': 1
})

census_test_df.sample(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
2100,31,Private,173473,Masters,14,1,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,45,United-States,>50K.
4559,53,Private,138022,Bachelors,13,1,Farming-fishing,Husband,White,Male,0,0,60,United-States,<=50K.
12804,50,Private,98815,Assoc-voc,11,1,Adm-clerical,Husband,White,Male,0,0,46,United-States,>50K.
4723,35,Private,251091,Masters,14,1,Adm-clerical,Husband,White,Male,0,0,38,Puerto-Rico,<=50K.
4357,46,Private,132919,HS-grad,9,1,Adm-clerical,Wife,White,Female,0,0,12,United-States,>50K.


#### Format 'income' column data to binary

In [ ]:
# Convert ' <=50K.' to '0' and ' >50K.' to '1'
census_test_df['income'] = census_test_df['income'].replace({
    ' <=50K.': 0, 
    ' >50K.': 1
})

census_test_df.sample(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
5830,39,Private,107991,Some-college,10,1,Exec-managerial,Husband,White,Male,0,0,45,United-States,1
8125,59,Private,201159,12th,8,0,Machine-op-inspct,Other-relative,White,Female,0,0,48,United-States,0
1289,47,Private,132912,Some-college,10,1,Exec-managerial,Husband,White,Male,0,0,55,United-States,1
9694,38,Self-emp-inc,298539,HS-grad,9,0,Exec-managerial,Not-in-family,White,Male,0,0,50,United-States,0
15803,37,Private,175614,10th,6,0,Other-service,Unmarried,White,Female,0,0,40,United-States,0


In [19]:
# The workclass, occupation, and native-country columns have a value '?' which means it is a missing value. 
# We will replace it with a NaN value
census_test_df['workclass'] = census_df['workclass'].replace(r'^\s*\?\s*$', np.nan, regex=True)
census_test_df['occupation'] = census_df['occupation'].replace(r'^\s*\?\s*$', np.nan, regex=True)
census_test_df['native-country'] = census_df['native-country'].replace(r'^\s*\?\s*$', np.nan, regex=True)

In [18]:
# Drop all records that contain missing values (NaN)
census_test_df.dropna(inplace=True)

In [19]:
# Check how many rows of census_test_df
census_test_df.shape

(15075, 15)

##### For the Test datasets, we removed 1205 null values that originally had ? sign.
##### We now work with 15,075 rows from originally 16,280 rows